This dataset is part of the [Farming Systems Project](https://www.ars.usda.gov/northeast-area/beltsville-md-barc/beltsville-agricultural-research-center/sustainable-agricultural-systems-laboratory/docs/farming-systems-project/) at USDA, Beltsville MD.  This data is not available online on the USDA
 website but can be found on my [GitHub](https://github.com/mmtokay/DATA606/tree/master/datasets).


The data is split in two files, one that contains crop information and other with weather data.

Crop file:
* Crop - wheat, corn or soybean           
* GrowingSeason - year crop was cultivated 
* SystemName - crop management (traditional: NT and CT; organic: Org2, Org3 and Org6')    
* GrainYield - grain yield measured in kg/ha     
* PlantingDate - date seeds were planted  
* HarvestDate - date crop was harvested


Weather file:
* Year 
* Julian Day 
* Month
* Day
* Date
* avgtTempC - average temperature in C
* maxTempC - maximum temperature in C
* minTempC - minimum temperature in C
* maxHumPct - maximum humidity in %
* minHumPct - minimum humidity in %
* avgRadWm-2 - average radiation in w/m2
* meanWindMs-1 - mean wind in m/s
* PrecipitationMm - precipitation/snow melt in mm

# Exploratory Data Analysis (Preliminary)


In [ ]:
import io
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
import warnings
import time
import pathlib
import seaborn as sns
import tensorflow as tf

from datetime import datetime, timedelta
from sklearn import linear_model
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression, RidgeClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, cross_val_predict, cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, PolynomialFeatures, RobustScaler, Normalizer, MinMaxScaler, StandardScaler, Binarizer
from sklearn.tree import DecisionTreeRegressor, export_graphviz
from sklearn.utils import shuffle
from time import time
from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)
warnings.simplefilter(action='ignore', category=FutureWarning)

# Crop Data
Import crop data file.

In [ ]:
data = pd.read_csv('./dataset/FSPGrainYieldsV3Clean.csv')
print(data.head())
print('\nData information.')
print(data.info())

Data contains 6 columns and 1113 rows.

Data conversion:
*   Convert GrainYield to numeric.
*   Convert PlantingDate and HarvestDate to datetime.



In [ ]:
# Convert from object to float64
data['GrainYield'] = pd.to_numeric(data.GrainYield, errors='coerce')

# Convert PlantingDate and HarvestDate from object to date
data['PlantingDate'] = pd.to_datetime(data.PlantingDate)
data['HarvestDate'] = pd.to_datetime(data.HarvestDate)

Let's check if there is any data missing on the dataset.  

In [ ]:
data.isna().sum()

I don't have harvest date for 71 measurements.  Harvest date is not critical because often times the crop is not harvest as soon as the crop is mature.  It is important to know how long each crop takes to mature on average.  I will create a new column called weekDuration to calculate the week duration between planting date and harvest date.

In [ ]:
# Calculate duration between PlantingDate and HarvestDate
data['weekDuration'] = data['HarvestDate'] - data['PlantingDate']
data['weekDuration'] = data['weekDuration']/np.timedelta64(1,'W')
print('\nCheck unique values for Crop, GrowingSeason and SystemName columns.\n')
print("Crop", data.Crop.unique())
print("\nGrowing Season", data.GrowingSeason.unique())
print("\nCrop Management Type", data.SystemName.unique())

There are duplicate values for SystemManagement because column values are case sensitive.  I will convert SystemName column to uppercase.

**Note:** There is no data for 1999 because it was a dry year and this experiment doesn't use irrigation, crops never matured.

In [ ]:
data['SystemName'] = data['SystemName'].str.upper()
print("\nCrop Management Type", data.SystemName.unique())

I will create a new column called SystemNameType for conventional and organic management.  These values will be generated from SystemName column. 

Conventional = NT and CT

Organic = ORG2, ORG3 and ORG6

In [ ]:
# 1 for conventional
# 0 for organic
data['SystemNameType'] = ((data.SystemName == "NT") | (data.SystemName == "CT")).map({True:'1', False:'0'})
# Drop SystemName column
data.drop('SystemName', axis=1, inplace=True)
data.head()

I will separate the data by crop: corn, soybean and wheat and I will display basic statistics for each crop.

# Corn dataset - Statistics

In [ ]:
data_corn = data.loc[data['Crop'] == "CRN"]
print(data_corn.describe(include="all"))
print('\nCorn Data Distribution by Year\n')
data_corn_grouped = data_corn.groupby(['GrowingSeason'], as_index=False).agg({'GrainYield': "count"})
print(data_corn_grouped.head())

In [ ]:
y_pos = np.arange(len(data_corn_grouped['GrowingSeason']))
plt.bar(y_pos, data_corn_grouped['GrainYield'], align='center')
plt.xticks(y_pos, data_corn_grouped['GrowingSeason'], rotation=45)
plt.ylabel('Number of Samples')
plt.title('Number of Corn Yield Samples per Year')
plt.show()

Corn Data Distribution by Crop Management Type

In [ ]:
data_corn_sys_grouped = data_corn.groupby(['SystemNameType'], as_index=False).agg({'GrainYield': "count"})
data_corn_sys_grouped.head()

In [ ]:
y_pos = np.arange(len(data_corn_sys_grouped['SystemNameType']))
plt.bar(y_pos, data_corn_sys_grouped['GrainYield'], align='center')
plt.xticks(y_pos, data_corn_sys_grouped['SystemNameType'], rotation=45)
plt.ylabel('Number of Samples')
plt.title('Number of Corn Yield Samples per Crop Management Type')
plt.show()

# Soybean dataset - Statistics

In [ ]:
data_soy = data.loc[data['Crop'] == "SOY"]
print(data_soy.describe(include="all"))
print('\nSoybean Data Distribution by Year\n')
data_soy_grouped = data_soy.groupby(['GrowingSeason'], as_index=False).agg({'GrainYield': "count"})
print(data_soy_grouped.head())

In [ ]:
y_pos = np.arange(len(data_soy_grouped['GrowingSeason']))
plt.bar(y_pos, data_soy_grouped['GrainYield'], align='center')
plt.xticks(y_pos, data_soy_grouped['GrowingSeason'], rotation=45)
plt.ylabel('Number of Samples')
plt.title('Number of Soybean Samples per Year')
plt.show()

Soybean Data Distribution by Crop Management Type

In [ ]:
data_soy_sys_grouped = data_soy.groupby(['SystemNameType'], as_index=False).agg({'GrainYield': "count"})
data_soy_sys_grouped.head()

In [ ]:
y_pos = np.arange(len(data_soy_sys_grouped['SystemNameType']))
plt.bar(y_pos, data_soy_sys_grouped['GrainYield'], align='center')
plt.xticks(y_pos, data_soy_sys_grouped['SystemNameType'], rotation=45)
plt.ylabel('Number of Samples')
plt.title('Number of Soybean Yield Samples per Crop Management Type')
plt.show()

# Wheat dataset - Statistics

In [ ]:
data_wheat = data.loc[data['Crop'] == "WHT"]
print(data_wheat.describe(include="all"))
print('\nWheat Data Distribution by Year\n')
data_wheat_grouped = data_wheat.groupby(['GrowingSeason'], as_index=False).agg({'GrainYield': "count"})
print(data_wheat_grouped.head())

In [ ]:
y_pos = np.arange(len(data_wheat_grouped['GrowingSeason']))
plt.bar(y_pos, data_wheat_grouped['GrainYield'], align='center')
plt.xticks(y_pos, data_wheat_grouped['GrowingSeason'], rotation=45)
plt.ylabel('Number of Samples')
plt.title('Number of Wheat Samples per Year')
plt.show()

Wheat Data Distribution by Crop Management Type

In [ ]:
data_wheat_sys_grouped = data_wheat.groupby(['SystemNameType'], as_index=False).agg({'GrainYield': "count"})
data_wheat_sys_grouped.head()

In [ ]:
y_pos = np.arange(len(data_wheat_sys_grouped['SystemNameType']))
plt.bar(y_pos, data_wheat_sys_grouped['GrainYield'], align='center')
plt.xticks(y_pos, data_wheat_sys_grouped['SystemNameType'], rotation=45)
plt.ylabel('Number of Samples')
plt.title('Number of Wheat Yield Samples per Crop Management Type')
plt.show()

# Weather Data

Import weather data.

In [ ]:
weather_data = pd.read_csv('./dataset/FSPWeather1996-2019V2.csv')
print(weather_data.head())
print('\nData information.\n')
print(weather_data.info())

Data contains 13 columns and 8762 rows. Convert Date to datetime.

In [ ]:
weather_data['Date'] = pd.to_datetime(weather_data.Date)

# Weather Data - Statistics

In [ ]:
weather_data.describe(include="all")

Let's check if there is any data missing on the dataset.

In [ ]:
weather_data.isna().sum()

There are some data missing.  Let's plot each variable to check for anomalies in the data.

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,5)
ax.plot(weather_data['Date'], weather_data['avgtTempC'])
ax.set(xlabel='Date', ylabel='Temperature (C)', title='Average Temperature')
ax.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,5)
ax.plot(weather_data['Date'], weather_data['maxTempC'])
ax.set(xlabel='Date', ylabel='Temperature (C)', title='Maximum Temperature')
ax.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,5)
ax.plot(weather_data['Date'], weather_data['minTempC'])
ax.set(xlabel='Date', ylabel='Temperature (C)', title='Minimum Temperature')
ax.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,5)
ax.plot(weather_data['Date'], weather_data['maxHumPct'])
ax.set(xlabel='Date', ylabel='Humidity (%)', title='Maximum Humidity')
ax.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,5)
ax.plot(weather_data['Date'], weather_data['minHumPct'])
ax.set(xlabel='Date', ylabel='Humidity (%)', title='Minimum Humidity')
ax.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,5)
ax.plot(weather_data['Date'], weather_data['avgRadWm-2'])
ax.set(xlabel='Date', ylabel='Radiation (W/m2)', title='Average Radiation')
ax.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,5)
ax.plot(weather_data['Date'], weather_data['meanWindMs-1'])
ax.set(xlabel='Date', ylabel='Wind (m/s)', title='Mean Wind')
ax.grid()
plt.show()

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(20,5)
ax.plot(weather_data['Date'], weather_data['PrecipitationMm'])
ax.set(xlabel='Date', ylabel='Precipitation (mm)', title='Precipitation')
ax.grid()
plt.show()

Looking at the graphics, I will not use average radiation because data is missing for years 2003-2008.  I need to check the anomalies for maximum humidity and mean wind and how this can be corrected.

My next step is to work on feature engineering, I will combine crop mangement type in two categories traditional and organic.  After I determine the average number of weeks that takes for each crop to mature I will calculate weather variables weekly average starting from planting date.

I will also calculate growing degree days (GDD) that "are used to estimate the growth and development of plants and insects during the growing season. The basic concept is that development will only occur if the temperature exceeds some minimum development threshold, or base temperature (TBASE). The base temperatures are determined experimentally and are different for each organism". [1]

GDD formula for corn and soybean:

GDD = (Daily Max Temp °C + Daily Min Temp °C) / 2 - 10

GDD formula wheat:

GDD = (Daily Max Temp °C + Daily Min Temp °C) / 2 - 4.4

# References

1. Explanation of Growing Degree Days, Midwestern Regional Climate Center, mrcc.illinois.edu/gismaps/info/gddinfo.htm.
